## Singleton Pattern

### Implement Singleton by Overriding `__new__`

In [2]:
class Singleton:
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super().__new__(cls)
        return cls.instance


s1 = Singleton()
s2 = Singleton()
print(s1)
print(s2)

**`__init__` will be called for multiple times because `__new__` returns an uninitialized object on which `__init__` will be called afterwards.**

We can avoid this later by using meta class.

In [5]:
class Singleton:
    def __new__(cls, *args, **kwargs):
        if not hasattr(cls, 'instance'):
            cls.instance = super().__new__(cls)
        return cls.instance


class ResourceIntensiveClass(Singleton):        
    def __init__(self, name):
        self.name = name
        print("The resource-intensive class {} is initialized".format(self.name))

    def __str__(self):
        return "Resource-intensive class {}".format(self.name)    

obj_1 = ResourceIntensiveClass("B")
obj_2 = ResourceIntensiveClass("C")
print(obj_1)
print(obj_2)

The resource-intensive class B is initialized
The resource-intensive class C is initialized
Resource-intensive class C
Resource-intensive class C


### Implement Lazy Singleton

There will be single instance for resource-intensive class, though Singleton could have multiple instances.

In [3]:
class Singleton:
    __instance = None

    def __init__(self):
        if not Singleton.__instance:
            print("Create object without initialization")
        else:
            print("Already have initialized object", Singleton.__instance)

    @classmethod
    def get_instance(cls, name):
        if not cls.__instance:
            cls.__instance = ResourceIntensiveClass(name)
        return cls.__instance

    
class ResourceIntensiveClass(Singleton):        
    def __init__(self, name):
        self.name = name
        print("The resource-intensive class {} is initialized".format(self.name))

    def __str__(self):
        return "Resource-intensive class {}: {}".format(self.name, id(self)) 

s1 = Singleton()
obj_1 = Singleton.get_instance('A')
s2 = Singleton()
obj_2 = Singleton.get_instance('B')
print(s1, s2)
print(obj_1, obj_2)

Create object without initialization
The resource-intensive class A is initialized
Already have initialized object Resource-intensive class A: 139956547891944
<__main__.Singleton object at 0x7f4a2c523390> <__main__.Singleton object at 0x7f4a2c523320>
Resource-intensive class A: 139956547891944 Resource-intensive class A: 139956547891944


### Implement Monostate/Borg Pattern

It does not matter whether class has only one instance. Important is all the instances share the same status.

In [9]:
class Brog:
    __shared_state = {}
    
    def __new__(cls, *args, **kwargs):
        obj = super().__new__(cls)
        obj.__dict__ = cls.__shared_state
        return obj
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __str__(self):
        return "Name: {}, Age: {} | (id={})".format(self.name, self.age, id(self))
    

b1 = Brog("Sun", 30)
print(b1)
b2 = Brog("Will", 30)
print(b1)
print(b2)

Name: Sun, Age: 30 | (id=139956547378592)
Name: Will, Age: 30 | (id=139956547378592)
Name: Will, Age: 30 | (id=139956547376352)


### Implement Singleton by Using Meta Class

`__init__` is invoked only once using meta class.

In [12]:
class MetaSingleton(type):
    __instances = {}
    
    def __call__(cls, *args, **kwargs):
        if cls not in cls.__instances:
            cls.__instances[cls] = super().__call__(*args, **kwargs)
        return cls.__instances[cls]
    
    
class ResourceIntensiveClass(metaclass=MetaSingleton):
    def __init__(self, name):
        self.name = name
        print("__init__ is invoked")
        
    def __str__(self):
        return "Resource-intensive Class {}({})".format(self.name, id(self))
    

s1 = ResourceIntensiveClass("A")
print(s1)
s2 = ResourceIntensiveClass("B")
print(s1)
print(s2)

__init__ is invoked
Resource-intensive Class A(139956547466520)
Resource-intensive Class A(139956547466520)
Resource-intensive Class A(139956547466520)


## Factory Pattern

### Simple Factory

In [6]:
from abc import ABCMeta, abstractmethod


class AbstractProduct(metaclass=ABCMeta):
    @abstractmethod
    def describe(self):
        pass


class ProductA(AbstractProduct):
    def describe(self):
        print("This is Product A")
        

class ProductB(AbstractProduct):
    def describe(self):
        print("This is Product B")
        
        
class Factory:
    def make_product(self, product):
        return eval(product)()
        

f = Factory()
p = f.make_product('ProductB')
p.describe()

This is Product B


### Factory Method

It is not necessary for client to care about which instance should be created. Just call factory method, and it will handle these things for you!

In [8]:
from abc import ABCMeta, abstractmethod


class AbstractCreator(metaclass=ABCMeta):
    def __init__(self):
        self.products = []

    @abstractmethod
    def factory_method(self):
        pass

    def get_products(self):
        for p in self.products:
            p.describe()

    def add_product(self, product):
        self.products.append(product)

class Creator1(AbstractCreator):
    def factory_method(self):
        self.add_product(ProductA())
        self.add_product(ProductB())
        print("Creator1 :")
        self.get_products()


class Creator2(AbstractCreator):
    def factory_method(self):
        self.add_product(ProductB())
        self.add_product(ProductC())
        print("Creator2 :")
        self.get_products()


class AbstractProduct(metaclass=ABCMeta):
    @abstractmethod
    def describe(self):
        pass


class ProductA(AbstractProduct):
    def describe(self):
        print("This is Product A")


class ProductB(AbstractProduct):
    def describe(self):
        print("This is Product B")


class ProductC(AbstractProduct):
    def describe(self):
        print("This is Product C")


c1 = Creator1()
c2 = Creator2()
c1.factory_method()
c2.factory_method()

Creator1 :
This is Product A
This is Product B
Creator2 :
This is Product B
This is Product C


### Abstract Factory

2 product instance, those are the same kind of product but created by different factory, can have different behavior.

In [6]:
from abc import ABCMeta, abstractmethod


class AbstractFactory(metaclass=ABCMeta):
    def __init__(self, name):
        self.name = name

    @abstractmethod
    def create_product_1(self):
        pass
    
    @abstractmethod
    def create_product_2(self):
        pass
    
    @abstractmethod
    def create_product_3(self):
        pass

    def describe(self):
        print(self.name + " is producing...")


class FactoryA(AbstractFactory):
    def create_product_1(self):
        return Product1_1()

    def create_product_2(self):
        return Product2_1()

    def create_product_3(self):
        return Product3_1()


class FactoryB(AbstractFactory):
    def create_product_1(self):
        return Product1_2()

    def create_product_2(self):
        return Product2_2()

    def create_product_3(self):
        return Product3_2()


class Product1(metaclass=ABCMeta):
    @abstractmethod
    def describe(self):
        pass


class Product2(metaclass=ABCMeta):
    @abstractmethod
    def do_say(self):
        pass


class Product3(metaclass=ABCMeta):
    @abstractmethod
    def do_walk(self):
        pass


class Product1_1(Product1):
    def describe(self):
        print("This is Product1-1")


class Product1_2(Product1):
    def describe(self):
        print("This is Product1-2")


class Product2_1(Product2):
    def do_say(self):
        print("I'm product2-1")


class Product2_2(Product2):
    def do_say(self):
        print("I'm product2-2")


class Product3_1(Product3):
    def do_walk(self):
        print("Product3-1 is walking at 5m/s")


class Product3_2(Product3):
    def do_walk(self):
        print("Product3-2 is walking at 10m/s")


f1 = FactoryA('Factory A')
f2 = FactoryB('Factory B')
for f in [f1, f2]:
    p1 = f.create_product_1()
    p2 = f.create_product_2()
    p3 = f.create_product_3()
    f.describe()
    p1.describe()
    p2.do_say()
    p3.do_walk()

Factory A is producing...
This is Product1-1
I'm product2-1
Product3-1 is walking at 5m/s
Factory B is producing...
This is Product1-2
I'm product2-2
Product3-2 is walking at 10m/s


## Observer Pattern

In [7]:
from abc import ABCMeta, abstractmethod


class Subject:
    def __init__(self, name):
        self.name = name
        self.__observers = []

    def add_observer(self, observer):
        self.__observers.append(observer)

    def detach_observer(self, observer):
        self.__observers.remove(observer)

    def update(self, content):
        for observer in self.__observers:
            observer.notify(content)


class Observer(metaclass=ABCMeta):
    def __init__(self, name):
        self.name = name

    def subscribe(self, subject):
        subject.add_observer(self)

    def unsubscribe(self, subject):
        subject.detach_observer(self)

    @abstractmethod
    def notify(self, content):
        pass


class Observer1(Observer):
    def notify(self, content):
        print("Dear {name}: Thank you for watching our channel! {content}".format(name=self.name, content=content))


class Observer2(Observer):
    def notify(self, content):
        print("Dear {name}: Thank you for reading our newspaper! {content}".format(name=self.name, content=content))


daily_news = Subject('Daily News')
observer_1 = Observer1('Jon')
observer_2 = Observer2('Wolfgang')
observer_3 = Observer2('Nomi')
observer_1.subscribe(daily_news)
observer_2.subscribe(daily_news)
observer_3.subscribe(daily_news)
daily_news.update('Today is the war of Winterfall in "The Game of Throne"!')
observer_1.unsubscribe(daily_news)
daily_news.update('"The Game of Throne" S08E03 is playing!')

Dear Jon: Thank you for watching our channel! Today is the war of Winterfall in "The Game of Throne"!
Dear Wolfgang: Thank you for reading our newspaper! Today is the war of Winterfall in "The Game of Throne"!
Dear Nomi: Thank you for reading our newspaper! Today is the war of Winterfall in "The Game of Throne"!
Dear Wolfgang: Thank you for reading our newspaper! "The Game of Throne" S08E03 is playing!
Dear Nomi: Thank you for reading our newspaper! "The Game of Throne" S08E03 is playing!
